In [1]:
import psycopg2
import boto3
import os
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


from dotenv import load_dotenv

from rds_connector import rds_connection
from rds_connector import RDSTablePull

from data_cleaning_utils import clean_empty_none

from query_package import get_query_package

# Load environment variables from .env file
load_dotenv()

# Access the secret value
username = os.getenv('USER')
password = os.getenv('PASS')
db = os.getenv('DB')
server = os.getenv('SERVER')

access = os.getenv("ACCESS")
secret = os.getenv("SECRET")

try:

    #Connect to AWS RDS Database
    conn, cursor = rds_connection(username,
                          password,
                          db,
                          server)
    
except Exception as e:
    
    #Raise Error
    raise Exception(f"AWS RDS CONNECTION ERROR: {e}")

# Pull the Query Package
query_package = get_query_package()

In [3]:
rds = RDSTablePull(conn = conn,
                   cursor = cursor,
                   query_package = query_package)

df = rds.query_to_df()

In [22]:
for item in rds.cleaning_versions:
    
    # Create File Name
    if item['Field'] == None:
        file_name = f"{item['Step']}"
    else:
        file_name = f"{item['Field']}: {item['Step']}"

    # Store DataFrame as CSV
    #data = item['Result'].to_csv(f"{file_name}.csv")

In [ ]:
# Bucket and file information
bucket_name = 'reporting-external'

# Create a S3 client
s3 = boto3.client('s3', aws_access_key_id=access, aws_secret_access_key=secret)

result = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')
